In [43]:
import py7zr
import pandas as pd
import os
import xml.etree.ElementTree as ET

# Step 1: Extract the .7z file
extract_folder = "path_to_extract_folder"
with py7zr.SevenZipFile("C:\\Users\\schen\\DB_NLP_Project\\ai.stackexchange.com.7z", mode='r') as z:
    z.extractall(extract_folder)

# Find the extracted file (assuming only one file is in the archive)
extracted_files = os.listdir(extract_folder)
if not extracted_files:
    raise ValueError("No files found in the extracted folder!")
data_path = os.path.join(extract_folder, extracted_files[0])

# Parsing the XML data
tree = ET.parse(data_path)
root = tree.getroot()

# Creating an empty list to store rows
data = []

# Extracting rows from XML
for row in root.findall('row'):
    data.append(row.attrib)

# Convert to DataFrame
df1 = pd.DataFrame(data)
print(df1.head())


  Id UserId      Name                     Date Class TagBased
0  1      4  Informed  2016-08-02T15:38:29.913     3    False
1  2      9  Informed  2016-08-02T15:39:20.227     3    False
2  3     16  Informed  2016-08-02T15:39:28.290     3    False
3  4     18  Informed  2016-08-02T15:39:40.377     3    False
4  5     10  Informed  2016-08-02T15:39:56.643     3    False


In [44]:
top_users = df['UserId'].value_counts().head(10)
print(top_users)

2444     286
8        137
1847     101
1641      93
4302      89
1671      83
145       71
18758     71
16909     70
16565     69
Name: UserId, dtype: int64


In [6]:
def lemmatize_words(words): # used
    lemmatized_words = []
    for word in words:
        tempList = []
        for word2 in word:
            tempList.append(wordlemmatizer.lemmatize(word2))
        lemmatized_words.append(tempList)
    return lemmatized_words

def uniqueWord(w): # used 
    w2=[]
    for word in w:
        tempList=[]
        for word2 in word:
            if tempList.count(word2)<1:
                    tempList.append(word2)
        w2.append(tempList)
    return w2

def remove_special_characters(text): # used
    regex = r'[^a-zA-Z0-9\s]'
    text = re.sub(regex,'',text)
    return text

def removeStopWord(word_text):  # used
    filtered_sentence = [] 
    stop_words = set(stopwords.words('english'))   
    for w in word_text:
        tempList=[]
        for x in w:
            if x.lower() not in stop_words: 
                tempList.append(x)
        filtered_sentence.append(tempList)
    return filtered_sentence   
def meanOfWord(model, sentence): # used
#     posValue=nltk.pos_tag(sentence)
    posList=['CD']
    nounList=['NN','NNP','NNS','NNPS']
    value=[]
    count=0
    noun=0
    for word in sentence:
        a=model.wv.most_similar(word)
        temp=[]
        for w in a:
            temp.append(w[1])
        posValue=nltk.pos_tag([word])
#         print(posValue)
        wordScore=np.mean(temp)
        if posValue[0][1] in posList:
            count=count+1
        else:
            valueIfNum=checkNum(word)
            count=count+valueIfNum
        if posValue[0][1] in nounList:
            noun=noun + .25
        value.append(wordScore)
    return np.mean(value)+count+noun

def checkNum(s):
    l= ['1','2','3','4','5','6','7','8','9','0']
    check =False

    for i in s:
        if i in l:
            check = True
            break
    if check == True:
        return 1
    else:
        return 0

Stopwords = set(stopwords.words('english'))
wordlemmatizer = WordNetLemmatizer()

score_list =[] #store date with sentence score and price

In [7]:
score=[]
for i, row in tweets_df.iterrows():
    text = row['Text']
    sentences = sent_tokenize(text) # 1: sent tokenize
    text_noSpecial_character = remove_special_characters(str(text)) # 2: remove special character:
    word_text = [[text_noSpecial_character for text_noSpecial_character in sentences.split()] for sentences in sentences] # 3: word token
    stop_text= removeStopWord(word_text) # 4: remove stop words
    unique_text= uniqueWord(stop_text)   # 5: remove duplicate words
    lemma_text = lemmatize_words(unique_text) # 6: lemmatization

    model = Word2Vec(lemma_text, min_count=1,sg=1)
    
    count = 0
    for index, sentence in enumerate(lemma_text):
        l = lemma_text.index(sentence)
        meanScore= meanOfWord(model,sentence)
#         print(str(labels[index])+ ":"+ str(sentence)+ str(meanScore) )
        if index == 0:
            count = count + meanScore
    
    if (i  + 1) % 100 == 0:
        WeekScore = count / 100
        score.append(WeekScore)
        count = 0
    
score_df = pd.DataFrame(score, columns=['SentenceScore'])

NameError: name 'tweets_df' is not defined